In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import cv2
import keras
import tensorflow
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras_unet_collection import models, base, utils,losses
import glob
from skimage import morphology
from skimage.measure import label
from PIL import Image
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,KFold
import time
import datetime


def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = keras.round(keras.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = keras.round(keras.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = keras.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = keras.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # keras.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + keras.epsilon())

    # return a single tensor value
    return recall

def load_test_data(test_npy_path):
    print('-'*30)
    print('load test images...')
    print('-'*30)

    imgs_test = cv2.imread(test_npy_path)
    imgs_test = imgs_test.astype('float32')
    imgs_test /= 255

    return imgs_test

def load_data(test_npy_path):
    imgs_test = np.array([load_test_data(test_npy_path)])
    print(imgs_test.shape)
    return imgs_test

In [2]:
def load_data2(npy_path):
    print('-'*30)
    print('load images...')
    print('-'*30)
    
    for i, path in enumerate(npy_path):
        print(path)
        train_npy_path = path.replace('_mask','')
        mask_npy_path = path
    
        imgs_tmp = np.load(train_npy_path)
        imgs_mask_tmp = np.load(mask_npy_path)
#         print(imgs_tmp.shape, imgs_mask_tmp.shape)
        
        if i==0:
            imgs = imgs_tmp
            imgs_mask = imgs_mask_tmp
            
        else:
            imgs = np.append(imgs, imgs_tmp, axis = 0)
            imgs_mask = np.append(imgs_mask, imgs_mask_tmp, axis = 0)
            
    print('-'*30)
    print('imgs : {} \nmasks : {}'.format(imgs.shape, imgs_mask.shape))    
    print('-'*30)
    
    imgs = imgs.astype('float32')
    imgs_mask = imgs_mask.astype('float32')
    print('img : ', imgs.max())
    print('mask : ',imgs_mask.max())

    print('-'*30)
    print('normalization start...')
    print('-'*30)
    
    imgs = imgs/255.0

    imgs_mask[imgs_mask<= 127] = 0
    imgs_mask[imgs_mask > 127] = 1

    print('img : ',imgs.max())
    print('mask : ',imgs_mask.max())

    return imgs, imgs_mask
from scipy import ndimage

def fill_hole(img):
    fill_img = ndimage.binary_fill_holes(img).astype(float)
    return fill_img

def fill_hole_cv(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img, contours, -1, (255,255,255), -1)
    return img

In [3]:
img_path = 'ROI/New/'
all_patient = np.array([i.replace('_HE.png', '') for i in os.listdir(img_path) if '_HE.png' in i])

In [4]:
np.array([os.path.basename(i).replace('_HE.png', '') for i in glob.glob("res/*/*.png") if '_HE.png' in i])

array(['1010', '1012', '1013', ..., '9500000', '970000', '9700000'],
      dtype='<U10')

In [5]:
kf = KFold(n_splits=5, random_state=5, shuffle=True)
fold = []
for train_index, test_index in kf.split(all_patient):
    train_patient, test_patient = all_patient[train_index.astype(int)], all_patient[test_index.astype(int)]    
    
    testset = [img_path+'{}_HE.png'.format(p) for p in test_patient]
    trainset = [img_path+'{}_HE.png'.format(p) for p in train_patient]

    fold.append([trainset, testset])

In [20]:
# num = 3
for foldnum in range(5):
    testset = fold[foldnum][1]

    # model = load_model('../4_result/exp3_HE/model/fold{}/exp1_lr0.01_epoch100.h5'.format(num), custom_objects={"dice_coef_loss": dice_coef_loss, 'recall':recall})
    model = load_model('6_result/exp_fold{}/model/best.h5'.format(foldnum), custom_objects={"dice": losses.dice, 'recall':recall})
    
    for test_path in testset:
        start = time.time()
        imgs_test = load_data(test_path)
        print(imgs_test.shape)
        imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)

        testset_name = test_path[4:]
        print(testset_name)
        name=testset_name
        ori_img_path = test_path

        pred_img_path = '6_result/exp_fold{}/pred_total/'.format(foldnum)
        pred_overimg_path = '6_result/exp_fold{}/pred_total_over/'.format(foldnum)
        pred_cut_path = '6_result/exp_fold{}/pred_crop/'.format(foldnum)
        if not os.path.isdir(pred_img_path):
            os.makedirs(pred_img_path)
        if not os.path.isdir(pred_overimg_path):
            os.makedirs(pred_overimg_path)
        if not os.path.isdir(pred_cut_path):
            os.makedirs(pred_cut_path)
        os.makedirs(os.path.dirname(pred_img_path+name), exist_ok=True)
        os.makedirs(os.path.dirname(pred_overimg_path+name), exist_ok=True)
        os.makedirs(os.path.dirname(pred_cut_path+name), exist_ok=True)
        
        pred_list=imgs_mask_test[0]
        print("prediction image")
        img = pred_list
#         img = imgs.astype('uint8')
        img[img <= 0.5] = 0
        img[img > 0.5] = 255
        img = img.astype('uint8')
        print(img.shape)
        img = fill_hole_cv(img)
        k = np.ones((2,2), np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, k)
        post_img = fill_hole_cv(img)
        post_img = post_img > 0
        post_img = morphology.remove_small_objects(post_img,min_size=5000)
        post_img = np.where(post_img,255,0)
        post_img = post_img.astype(np.uint8)
        
        print(post_img.shape)
        cv2.imwrite(pred_img_path+name, post_img)
        post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)
 
        ori_img = load_img(ori_img_path)
        ori_img = img_to_array(ori_img)
        ori_img[:,:,0] = post_img[:,:,0]+ori_img[:,:,0]
        ori_img[:,:,0][ori_img[:,:,0]>=255]=255
        ori_img = array_to_img(ori_img)

        ori_img.save(pred_overimg_path+name)
        
        ori_img = load_img(ori_img_path)
        ori_img = img_to_array(ori_img)
        
        crop_img = np.where(post_img != 0,ori_img,0)
        crop_img = array_to_img(crop_img)
        crop_img.save(pred_cut_path+name)
        
        cp1 = time.time()
        sec = (cp1 - start) 
        result = datetime.timedelta(seconds=sec) 
        print(result)
        


------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 249ms/step
New/1059_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.531476
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/1138_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.271321
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
New/840_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.261551
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/543_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.231174
------------------------------
load t

0:00:00.288551
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
New/609_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.210362
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/610_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.262508
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/627_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.264675
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 18ms/step
New/641_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.281548
-------------------------

(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 18ms/step
New/796_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.233534
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/798_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.236171
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/801_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.289643
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/817_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.365995
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [=

1/1 [==============================] - 0s 21ms/step
New/1061_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.272296
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/1066_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.284652
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/1067_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.291049
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/1069_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.266623
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================]

(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/201830000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.259087
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/201950000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.347159
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/201960000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.236761
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
New/201990000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.282459
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)

...

1/1 [==============================] - 0s 22ms/step
New/1590000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.252399
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/1050000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.345549
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 22ms/step
New/9700000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.286004
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/15200000_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.279629
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==================

(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/608_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.252962
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/612_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.264828
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/613_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.293705
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/618_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.240613
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [=

1/1 [==============================] - 0s 19ms/step
New/828_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.795547
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/838_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.253820
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/847_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.253778
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 18ms/step
New/849_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.242499
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0

(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 18ms/step
New/5_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.282952
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/14_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.177986
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 20ms/step
New/17_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.246806
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/22_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.257860
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [======

1/1 [==============================] - 0s 19ms/step
New/413_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.232666
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 21ms/step
New/419_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.331836
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 19ms/step
New/433_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.245567
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0s 31ms/step
New/436_HE.png
prediction image
(512, 512, 1)
(512, 512)
0:00:00.388799
------------------------------
load test images...
------------------------------
(1, 512, 512, 3)
(1, 512, 512, 3)
1/1 [==============================] - 0

In [ ]:
post_img.shape

In [ ]:


#     pred_img_path = '../4_result/exp3_HE/pred/fold{}/pred_img/{}/'.format(num,testset_name)
#     pred_overimg_path = '../4_result/exp3_HE/pred/fold{}/pred_img_over/{}/'.format(num,testset_name)
    

    pred_list=imgs_mask_test[0]
    print("prediction image")
    imgs = pred_list
    img = imgs.astype('uint8')
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    print(img.shape)
    img = fill_hole_cv(img)
    k = np.ones((2,2), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, k)
    post_img = fill_hole_cv(img)
#     print(post_img.max())
    cv2.imwrite(pred_img_path+name, post_img)
    post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)

    ori_img = load_img(ori_img_path)
    ori_img = img_to_array(ori_img)
    ori_img[:,:,0] = post_img[:,:,0]+ori_img[:,:,0]
    ori_img[:,:,0][ori_img[:,:,0]>=255]=255
    ori_img = array_to_img(ori_img)

    ori_img.save(pred_overimg_path+name)
    cp1 = time.time()
    sec = (cp1 - start) 
    result = datetime.timedelta(seconds=sec) 
    print(result)


# for j in range(len(testset)):
#     cp2 = time.time()
#     print(testset[j])
#     true_list=np.load(testset[j])
#     true_list=true_list.astype('float32')
#     # true_list = true_list/255.0
#     true_list[true_list > 0.5] = 1
#     true_list[true_list <= 0.5] = 0
#     print(true_list.shape)


#     imgs_test = load_data(testset[j].replace('_mask', ''))
#     print(imgs_test.shape)
#     pred_list = model.predict(imgs_test, batch_size=1, verbose=1)
#     # pred_list=imgs_mask_test
#     pred_list[pred_list > 0.5] = 1
#     pred_list[pred_list <= 0.5] = 0
#     print(pred_list.shape)

#     # for i in range(pred_list.shape[0]):
#     #     pred = pred_list[i].astype('uint8')
#     #     pred[pred <= 0.5] = 0
#     #     pred[pred > 0.5] = 255
#     # #     pred = fill_hole_cv(pred)
#     #     pred_list[i]=pred

#     # pred_list[pred_list > 127] = 1
#     # pred_list[pred_list <= 127] = 0

#     sensitivity=[]
#     specificity=[]
#     acc=[]
#     dsc=[]

#     for i in range(len(true_list)):
#         yt=true_list[i].flatten()
#         yp=pred_list[i].flatten()
#         mat=confusion_matrix(yt,yp)
#         if len(mat) == 2:
#             ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
#             st=mat[1,1]/(mat[1,0]+mat[1,1])
#             sp=mat[0,0]/(mat[0,1]+mat[0,0])
#             if mat[1,0]+mat[1,1] == 0:
#                 specificity.append(sp)
#                 acc.append(ac)
#             else:
#                 sensitivity.append(st)  
#                 specificity.append(sp)
#                 acc.append(ac)
#         else:
#             specificity.append(1)
#             acc.append(1)

#     for i in range(len(true_list)):
#         yt=true_list[i]
#         yp=pred_list[i]
#         if np.sum(yt) != 0 and np.sum(yp) != 0:
#             dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
#             dsc.append(dice)

#     print("complete")      
#     print("acc avg : {0:0.4f}".format(np.mean(acc)))
#     print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
#     print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
#     print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
#     end = time.time()
#     sec = (end - cp2) 
#     result = datetime.timedelta(seconds=sec) 
#     print(result)
#         print('-'*30)
#         print("sensitivity min:",np.min(sensitivity))
#         print("specificity min:",np.min(specificity))
#         print("dsc min:",np.min(dsc))
#         print("acc min:",np.min(acc))
#         print('-'*30)
#         print("sensitivity max:",np.max(sensitivity))
#         print("specificity max:",np.max(specificity))
#         print("dsc max:",np.max(dsc))
#         print("acc max:",np.max(acc))
#     num += 1

In [ ]:
# for test_path in testset:
#     start = time.time()
#     imgs_test = load_data(test_path)
#     print(imgs_test.shape)
#     imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)

#     testset_name = test_path.split('/')[-1][:-7]
#     print(testset_name)
#     name='{}.png'.format(testset_name)
#     ori_img_path = "ROI/New/{}".format(test_path.split('/')[-1])


# #     pred_img_path = '../4_result/exp3_HE/pred/fold{}/pred_img/{}/'.format(num,testset_name)
# #     pred_overimg_path = '../4_result/exp3_HE/pred/fold{}/pred_img_over/{}/'.format(num,testset_name)
#     pred_img_path = '4_result/exp/pred_img/'
#     pred_overimg_path = '4_result/exp/pred_img_over/'
#     if not os.path.isdir(pred_img_path):
#         os.makedirs(pred_img_path)
#     if not os.path.isdir(pred_overimg_path):
#         os.makedirs(pred_overimg_path)

#     pred_list=imgs_mask_test[0]
#     print("prediction image")
#     imgs = pred_list
#     img = imgs.astype('uint8')
#     img[img <= 0.5] = 0
#     img[img > 0.5] = 255
#     print(img.shape)
#     img = fill_hole_cv(img)
#     k = np.ones((2,2), np.uint8)
#     img = cv2.morphologyEx(img, cv2.MORPH_OPEN, k)
#     post_img = fill_hole_cv(img)
# #     print(post_img.max())
#     cv2.imwrite(pred_img_path+name, post_img)
#     post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)

#     ori_img = load_img(ori_img_path)
#     ori_img = img_to_array(ori_img)
#     ori_img[:,:,0] = post_img[:,:,0]+ori_img[:,:,0]
#     ori_img[:,:,0][ori_img[:,:,0]>=255]=255
#     ori_img = array_to_img(ori_img)

#     ori_img.save(pred_overimg_path+name)
#     cp1 = time.time()
#     sec = (cp1 - start) 
#     result = datetime.timedelta(seconds=sec) 
#     print(result)


# # for j in range(len(testset)):
# #     cp2 = time.time()
# #     print(testset[j])
# #     true_list=np.load(testset[j])
# #     true_list=true_list.astype('float32')
# #     # true_list = true_list/255.0
# #     true_list[true_list > 0.5] = 1
# #     true_list[true_list <= 0.5] = 0
# #     print(true_list.shape)


# #     imgs_test = load_data(testset[j].replace('_mask', ''))
# #     print(imgs_test.shape)
# #     pred_list = model.predict(imgs_test, batch_size=1, verbose=1)
# #     # pred_list=imgs_mask_test
# #     pred_list[pred_list > 0.5] = 1
# #     pred_list[pred_list <= 0.5] = 0
# #     print(pred_list.shape)

# #     # for i in range(pred_list.shape[0]):
# #     #     pred = pred_list[i].astype('uint8')
# #     #     pred[pred <= 0.5] = 0
# #     #     pred[pred > 0.5] = 255
# #     # #     pred = fill_hole_cv(pred)
# #     #     pred_list[i]=pred

# #     # pred_list[pred_list > 127] = 1
# #     # pred_list[pred_list <= 127] = 0

# #     sensitivity=[]
# #     specificity=[]
# #     acc=[]
# #     dsc=[]

# #     for i in range(len(true_list)):
# #         yt=true_list[i].flatten()
# #         yp=pred_list[i].flatten()
# #         mat=confusion_matrix(yt,yp)
# #         if len(mat) == 2:
# #             ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
# #             st=mat[1,1]/(mat[1,0]+mat[1,1])
# #             sp=mat[0,0]/(mat[0,1]+mat[0,0])
# #             if mat[1,0]+mat[1,1] == 0:
# #                 specificity.append(sp)
# #                 acc.append(ac)
# #             else:
# #                 sensitivity.append(st)  
# #                 specificity.append(sp)
# #                 acc.append(ac)
# #         else:
# #             specificity.append(1)
# #             acc.append(1)

# #     for i in range(len(true_list)):
# #         yt=true_list[i]
# #         yp=pred_list[i]
# #         if np.sum(yt) != 0 and np.sum(yp) != 0:
# #             dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
# #             dsc.append(dice)

# #     print("complete")      
# #     print("acc avg : {0:0.4f}".format(np.mean(acc)))
# #     print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
# #     print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
# #     print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
# #     end = time.time()
# #     sec = (end - cp2) 
# #     result = datetime.timedelta(seconds=sec) 
# #     print(result)
# #         print('-'*30)
# #         print("sensitivity min:",np.min(sensitivity))
# #         print("specificity min:",np.min(specificity))
# #         print("dsc min:",np.min(dsc))
# #         print("acc min:",np.min(acc))
# #         print('-'*30)
# #         print("sensitivity max:",np.max(sensitivity))
# #         print("specificity max:",np.max(specificity))
# #         print("dsc max:",np.max(dsc))
# #         print("acc max:",np.max(acc))
# #     num += 1

In [ ]:
test_patient

In [ ]:
fold[0]['testset']

In [ ]:
testset

In [ ]:
if 'Thumbs.db' in name_list:
        print('aaa')

In [ ]:
imgs_path = sorted(glob.glob('../testpred2/pred_img/*/*.png'))

post_img_path = '../testpred2/post_img/'


    
for i, img_path in enumerate(imgs_path):
    img_name = img_path[img_path.rindex('/')+1:-4]
#     print(img_name)
    folder = img_path.split('/')[4]
    post_img_path = os.path.dirname(img_path).replace("pred_img","pred_post")
    if not os.path.isdir(post_img_path):
        os.makedirs(post_img_path)
#     print(img_name)

#     if i==10:
#         break
    img = cv2.imread(img_path,0)
    kernel = np.ones((4, 4), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
#     opening = opening>0
#     post_img = morphology.remove_small_objects(opening, min_size=200, connectivity=4)
    post_img = Image.fromarray(close)
    post_img.save(post_img_path+"/{}_post.png".format(img_name))

In [ ]:
img_path[img_path.rindex('/')+1:-4]